In [ ]:
from transformers import OPTForCausalLM, AutoTokenizer
import torch
from torch import nn

model = OPTForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
prompt = "The meaning"
kwargs = tokenizer(prompt, return_tensors='pt').to(device)
relu = nn.ReLU()

def shift_right(attention_mask):
    batch_size, seq_length = attention_mask.size()
    output_mask = torch.zeros(batch_size, seq_length + 1, dtype=attention_mask.dtype)
    output_mask[:, :-1] = attention_mask    
    output_mask[:, -1] = 1
    return output_mask

In [ ]:
N = 6
output_vectors = []
output= kwargs["input_ids"]

for _ in range(N):
    with torch.no_grad():
        outputs = model(input_ids=output, output_hidden_states=True)
        
    next_token = torch.multinomial(torch.softmax(outputs.logits[:, -1, :], dim=-1), 1)
    output = torch.cat([output, next_token], dim=-1)
    
    print(tokenizer.decode(output[0], skip_special_tokens=True))
    
    logits = (
        outputs.logits
        * kwargs["attention_mask"].unsqueeze(-1)
    )
    logits = torch.log1p(relu(logits))
    
    print(logits)
    lex_weights =torch.max(logits, dim=1).values
    kwargs["attention_mask"] = shift_right(kwargs["attention_mask"])
    print("attention_mask ", kwargs["attention_mask"])
    output_vectors.append(lex_weights)